In [6]:
##### file path
### input
# data_set keys and lebels
path_df_part_1_uic_label = "mobile/df_part_1_uic_label.csv"
path_df_part_2_uic_label = "mobile/df_part_2_uic_label.csv"
path_df_part_3_uic       = "mobile/df_part_3_uic.csv"

# data_set features
path_df_part_1_U   = "mobile/feature/df_part_1_U.csv"  
path_df_part_1_I   = "mobile/feature/df_part_1_I.csv"
path_df_part_1_C   = "mobile/feature/df_part_1_C.csv"
path_df_part_1_IC  = "mobile/feature/df_part_1_IC.csv"
path_df_part_1_UI  = "mobile/feature/df_part_1_UI.csv"
path_df_part_1_UC  = "mobile/feature/df_part_1_UC.csv"

path_df_part_2_U   = "mobile/feature/df_part_2_U.csv"  
path_df_part_2_I   = "mobile/feature/df_part_2_I.csv"
path_df_part_2_C   = "mobile/feature/df_part_2_C.csv"
path_df_part_2_IC  = "mobile/feature/df_part_2_IC.csv"
path_df_part_2_UI  = "mobile/feature/df_part_2_UI.csv"
path_df_part_2_UC  = "mobile/feature/df_part_2_UC.csv"

path_df_part_3_U   = "mobile/feature/df_part_3_U.csv"  
path_df_part_3_I   = "mobile/feature/df_part_3_I.csv"
path_df_part_3_C   = "mobile/feature/df_part_3_C.csv"
path_df_part_3_IC  = "mobile/feature/df_part_3_IC.csv"
path_df_part_3_UI  = "mobile/feature/df_part_3_UI.csv"
path_df_part_3_UC  = "mobile/feature/df_part_3_UC.csv"

### out file

### intermediate file
# data partition with diffferent label
path_df_part_2_uic_label_0 = "mobile/gbdt/k_means_subsample/df_part_2_uic_label_0.csv"
path_df_part_2_uic_label_1 = "mobile/gbdt/k_means_subsample/df_part_2_uic_label_1.csv"

# training set keys uic-label with k_means clusters' label
# path_df_part_1_uic_label_cluster = "../../data/mobile/gbdt/k_means_subsample/df_part_1_uic_label_cluster.csv"
path_df_part_2_uic_label_cluster = "mobile/gbdt/k_means_subsample/df_part_2_uic_label_cluster.csv"

# scalers for data standardization store as python pickle
# for each part's features
# path_df_part_1_scaler = "../../data/mobile/gbdt/k_means_subsample/df_part_1_scaler"
path_df_part_2_scaler = "mobile/gbdt/k_means_subsample/df_part_2_scaler"

import pandas as pd
import numpy as np

In [7]:
def df_read(path, mode = 'r'):
    '''the definition of dataframe loading function 
    '''
    path_df = open(path, mode)
    try:     df = pd.read_csv(path_df, index_col = False)
    finally: path_df.close()
    return   df
# df_part_1_uic_label = df_read(path_df_part_1_uic_label)  # loading total keys
df_part_2_uic_label = df_read(path_df_part_2_uic_label)
# df_part_1_uic_label_0 = df_part_1_uic_label[df_part_1_uic_label['label'] == 0]
# df_part_1_uic_label_1 = df_part_1_uic_label[df_part_1_uic_label['label'] == 1]
df_part_2_uic_label_0 = df_part_2_uic_label[df_part_2_uic_label['label'] == 0]
df_part_2_uic_label_1 = df_part_2_uic_label[df_part_2_uic_label['label'] == 1]
# df_part_1_uic_label_0.to_csv(path_df_part_1_uic_label_0, index=False)
# df_part_1_uic_label_1.to_csv(path_df_part_1_uic_label_1, index=False)
df_part_2_uic_label_0.to_csv(path_df_part_2_uic_label_0, index=False)
df_part_2_uic_label_1.to_csv(path_df_part_2_uic_label_1, index=False)

In [8]:
#######################################################################
'''Step 2: clustering on negative sub-set
    clusters number ~ 35, using mini-batch-k-means
'''

# clustering based on sklearn
from sklearn import preprocessing
from sklearn.cluster import MiniBatchKMeans
import pickle

In [9]:
##### part_2 #####
# loading features
df_part_2_U  = df_read(path_df_part_2_U )   
df_part_2_I  = df_read(path_df_part_2_I )
df_part_2_C  = df_read(path_df_part_2_C )
df_part_2_IC = df_read(path_df_part_2_IC)
df_part_2_UI = df_read(path_df_part_2_UI)
df_part_2_UC = df_read(path_df_part_2_UC)

# process by chunk as ui-pairs size is too big

# for get scale transform mechanism to large scale of data
scaler_2 = preprocessing.StandardScaler()
batch = 0
for df_part_2_uic_label_0 in pd.read_csv(open(path_df_part_2_uic_label_0, 'r'), chunksize=150000): 
    try:
        # construct of part_1's sub-training set
        train_data_df_part_2 = pd.merge(df_part_2_uic_label_0, df_part_2_U, how='left', on=['user_id'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_I,  how='left', on=['item_id'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_C,  how='left', on=['item_category'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_IC, how='left', on=['item_id','item_category'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_UI, how='left', on=['user_id','item_id','item_category','label'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_UC, how='left', on=['user_id','item_category'])

        train_X_2 = train_data_df_part_2.as_matrix(['u_b1_count_in_6','u_b2_count_in_6','u_b3_count_in_6','u_b4_count_in_6','u_b_count_in_6', 
                                                    'u_b1_count_in_3','u_b2_count_in_3','u_b3_count_in_3','u_b4_count_in_3','u_b_count_in_3', 
                                                    'u_b1_count_in_1','u_b2_count_in_1','u_b3_count_in_1','u_b4_count_in_1','u_b_count_in_1', 
                                                    'u_b4_rate',
                                                    'i_u_count_in_6','i_u_count_in_3','i_u_count_in_1',
                                                    'i_b1_count_in_6','i_b2_count_in_6','i_b3_count_in_6','i_b4_count_in_6','i_b_count_in_6', 
                                                    'i_b1_count_in_3','i_b2_count_in_3','i_b3_count_in_3','i_b4_count_in_3','i_b_count_in_3',
                                                    'i_b1_count_in_1','i_b2_count_in_1','i_b3_count_in_1','i_b4_count_in_1','i_b_count_in_1', 
                                                    'i_b4_rate',
                                                    'c_b1_count_in_6','c_b2_count_in_6','c_b3_count_in_6','c_b4_count_in_6','c_b_count_in_6',
                                                    'c_b1_count_in_3','c_b2_count_in_3','c_b3_count_in_3','c_b4_count_in_3','c_b_count_in_3',
                                                    'c_b1_count_in_1','c_b2_count_in_1','c_b3_count_in_1','c_b4_count_in_1','c_b_count_in_1',
                                                    'c_b4_rate',
                                                    'ic_u_rank_in_c','ic_b_rank_in_c','ic_b4_rank_in_c', 
                                                    'ui_b1_count_in_6','ui_b2_count_in_6','ui_b3_count_in_6','ui_b4_count_in_6','ui_b_count_in_6',
                                                    'ui_b1_count_in_3','ui_b2_count_in_3','ui_b3_count_in_3','ui_b4_count_in_3','ui_b_count_in_3',
                                                    'ui_b1_count_in_1','ui_b2_count_in_1','ui_b3_count_in_1','ui_b4_count_in_1','ui_b_count_in_1', 
                                                    'ui_b_count_rank_in_u','ui_b_count_rank_in_uc',
                                                    'uc_b1_count_in_6','uc_b2_count_in_6','uc_b3_count_in_6','uc_b4_count_in_6','uc_b_count_in_6', 
                                                    'uc_b1_count_in_3','uc_b2_count_in_3','uc_b3_count_in_3','uc_b4_count_in_3','uc_b_count_in_3', 
                                                    'uc_b1_count_in_1','uc_b2_count_in_1','uc_b3_count_in_1','uc_b4_count_in_1','uc_b_count_in_1',
                                                    'uc_b_count_rank_in_u'])
        # fit the scaler
        scaler_2.partial_fit(train_X_2)
        
        batch += 1
        print('chunk %d done.' %batch) 
        
    except StopIteration:
        print("finish.")
        break 

# initial clusters
mbk_2 = MiniBatchKMeans(init='k-means++', n_clusters=1000, batch_size=500, reassignment_ratio=10**-4)  

# process by chunk as ui-pairs size is too big
batch = 0
classes_2 = []
for df_part_2_uic_label_0 in pd.read_csv(open(path_df_part_2_uic_label_0, 'r'), chunksize=15000): 
    try:
        # construct of part_1's sub-training set
        train_data_df_part_2 = pd.merge(df_part_2_uic_label_0, df_part_2_U, how='left', on=['user_id'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_I,  how='left', on=['item_id'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_C,  how='left', on=['item_category'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_IC, how='left', on=['item_id','item_category'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_UI, how='left', on=['user_id','item_id','item_category','label'])
        train_data_df_part_2 = pd.merge(train_data_df_part_2, df_part_2_UC, how='left', on=['user_id','item_category'])
        
        train_X_2 = train_data_df_part_2.as_matrix(['u_b1_count_in_6','u_b2_count_in_6','u_b3_count_in_6','u_b4_count_in_6','u_b_count_in_6', 
                                                    'u_b1_count_in_3','u_b2_count_in_3','u_b3_count_in_3','u_b4_count_in_3','u_b_count_in_3', 
                                                    'u_b1_count_in_1','u_b2_count_in_1','u_b3_count_in_1','u_b4_count_in_1','u_b_count_in_1', 
                                                    'u_b4_rate',
                                                    'i_u_count_in_6','i_u_count_in_3','i_u_count_in_1',
                                                    'i_b1_count_in_6','i_b2_count_in_6','i_b3_count_in_6','i_b4_count_in_6','i_b_count_in_6', 
                                                    'i_b1_count_in_3','i_b2_count_in_3','i_b3_count_in_3','i_b4_count_in_3','i_b_count_in_3',
                                                    'i_b1_count_in_1','i_b2_count_in_1','i_b3_count_in_1','i_b4_count_in_1','i_b_count_in_1', 
                                                    'i_b4_rate',
                                                    'c_b1_count_in_6','c_b2_count_in_6','c_b3_count_in_6','c_b4_count_in_6','c_b_count_in_6',
                                                    'c_b1_count_in_3','c_b2_count_in_3','c_b3_count_in_3','c_b4_count_in_3','c_b_count_in_3',
                                                    'c_b1_count_in_1','c_b2_count_in_1','c_b3_count_in_1','c_b4_count_in_1','c_b_count_in_1',
                                                    'c_b4_rate',
                                                    'ic_u_rank_in_c','ic_b_rank_in_c','ic_b4_rank_in_c', 
                                                    'ui_b1_count_in_6','ui_b2_count_in_6','ui_b3_count_in_6','ui_b4_count_in_6','ui_b_count_in_6',
                                                    'ui_b1_count_in_3','ui_b2_count_in_3','ui_b3_count_in_3','ui_b4_count_in_3','ui_b_count_in_3',
                                                    'ui_b1_count_in_1','ui_b2_count_in_1','ui_b3_count_in_1','ui_b4_count_in_1','ui_b_count_in_1', 
                                                    'ui_b_count_rank_in_u','ui_b_count_rank_in_uc',
                                                    'uc_b1_count_in_6','uc_b2_count_in_6','uc_b3_count_in_6','uc_b4_count_in_6','uc_b_count_in_6', 
                                                    'uc_b1_count_in_3','uc_b2_count_in_3','uc_b3_count_in_3','uc_b4_count_in_3','uc_b_count_in_3', 
                                                    'uc_b1_count_in_1','uc_b2_count_in_1','uc_b3_count_in_1','uc_b4_count_in_1','uc_b_count_in_1',
                                                    'uc_b_count_rank_in_u'])
        # feature standardization
        standardized_train_X_2 = scaler_2.transform(train_X_2)
        
        # fit clustering model
        mbk_2.partial_fit(standardized_train_X_2)
        classes_2 = np.append(classes_2, mbk_2.labels_)
        
        batch += 1
        print('chunk %d done.' %batch) 
        
    except StopIteration:
        print(" ------------ k-means finished on part 2 ------------.")
        break 

/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


chunk 1 done.
chunk 2 done.
chunk 3 done.
chunk 4 done.
chunk 5 done.
chunk 6 done.
chunk 7 done.
chunk 8 done.
chunk 9 done.
chunk 10 done.
chunk 11 done.
chunk 12 done.
chunk 13 done.


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:94: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


chunk 1 done.
chunk 2 done.
chunk 3 done.
chunk 4 done.
chunk 5 done.
chunk 6 done.
chunk 7 done.
chunk 8 done.
chunk 9 done.
chunk 10 done.
chunk 11 done.
chunk 12 done.
chunk 13 done.
chunk 14 done.
chunk 15 done.
chunk 16 done.
chunk 17 done.
chunk 18 done.
chunk 19 done.
chunk 20 done.
chunk 21 done.
chunk 22 done.
chunk 23 done.
chunk 24 done.
chunk 25 done.
chunk 26 done.
chunk 27 done.
chunk 28 done.
chunk 29 done.
chunk 30 done.
chunk 31 done.
chunk 32 done.
chunk 33 done.
chunk 34 done.
chunk 35 done.
chunk 36 done.
chunk 37 done.
chunk 38 done.
chunk 39 done.
chunk 40 done.
chunk 41 done.
chunk 42 done.
chunk 43 done.
chunk 44 done.
chunk 45 done.
chunk 46 done.
chunk 47 done.
chunk 48 done.
chunk 49 done.
chunk 50 done.
chunk 51 done.
chunk 52 done.
chunk 53 done.
chunk 54 done.
chunk 55 done.
chunk 56 done.
chunk 57 done.
chunk 58 done.
chunk 59 done.
chunk 60 done.
chunk 61 done.
chunk 62 done.
chunk 63 done.
chunk 64 done.
chunk 65 done.
chunk 66 done.
chunk 67 done.
chun

In [10]:
pickle.dump(scaler_2, open(path_df_part_2_scaler,'wb'))
# add a new attr for keys
# df_part_1_uic_label_0 = df_read(path_df_part_1_uic_label_0)
# df_part_1_uic_label_1 = df_read(path_df_part_1_uic_label_1)
df_part_2_uic_label_0 = df_read(path_df_part_2_uic_label_0)
df_part_2_uic_label_1 = df_read(path_df_part_2_uic_label_1)
    
# df_part_1_uic_label_0['class'] = classes_1.astype('int') + 1
# df_part_1_uic_label_1['class'] = 0
df_part_2_uic_label_0['class'] = classes_2.astype('int') + 1
df_part_2_uic_label_1['class'] = 0

# df_part_1_uic_label_class = pd.concat([df_part_1_uic_label_0, df_part_1_uic_label_1])
df_part_2_uic_label_class = pd.concat([df_part_2_uic_label_0, df_part_2_uic_label_1])
   
# df_part_1_uic_label_class.to_csv(path_df_part_1_uic_label_cluster, index=False)
df_part_2_uic_label_class.to_csv(path_df_part_2_uic_label_cluster, index=False)